In [1]:
import numpy as np
from scipy.signal import StateSpace, lsim
import matplotlib.pyplot as plt
from numpy import linalg as LA
import cvxpy as cp

In [2]:

a=np.random.rand(2)
b=np.random.rand(2)
a,b

Q=np.outer(a,a)
R=np.outer(b,b)

pd1=0.5*np.random.rand(1)
pd2=0.5*np.random.rand(1)

I=np.array([[1,0],[0,1]])

Q=Q+pd1*I
R=R+pd2*I



def SDP_solver(At,Bt,Vt):
    Vtinv=np.linalg.inv(Vt)
    Sigma=cp.Variable((n,n))
    sxx=cp.Variable((2,2))
    suu=cp.Variable((2,2))
    constraints=[Sigma>>0]
    bigI=np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
    ABt=np.column_stack((At,Bt))
    #Cost_Coefficient=np.array([[Q,[[1,0],[0,1]]],[[[1,0],[0,1]],R]])
    I=np.array([[1,0],[0,1]])
    
    Left=np.array([[1,0,0,0],[0,1,0,0]])
    Right=np.array([[1,0],[0,1],[0,0],[0,0]])
    
    constraints+=[ sxx  >> (((ABt)@Sigma)@(ABt.T))+W-mu*cp.trace((Sigma.T)@(Vtinv))*I ]
    
    constraints+=[ sxx == (Left@Sigma)@Right  ]
    
    Lefty=np.array([[0,0,1,0],[0,0,0,1]])
    Righty=np.array([[0,0],[0,0],[1,0],[0,1]])
    
    constraints+=[ suu == (Lefty@Sigma)@Righty  ]
    
    prob = cp.Problem(cp.Minimize(cp.trace(sxx.T @ Q + suu.T @ R)),constraints)
    prob.solve()
    return (Sigma.value)


In [3]:
At=np.random.rand(2,2)
Bt=np.random.rand(2,2)
Vt=np.random.rand(4,4)
W=np.random.rand(2,2)
mu=2
n=4

In [4]:
SDP_solver(At,Bt,Vt)

WARN: m less than n, problem likely degenerate


array([[ 5.39840036e-13,  6.50231753e-02, -4.80632872e-02,
         5.34460957e-02],
       [-6.50231753e-02,  1.79152189e-11,  5.24604342e-02,
        -7.23712055e-02],
       [ 4.80632872e-02, -5.24604342e-02,  4.29378371e-12,
         4.53396541e-02],
       [-5.34460957e-02,  7.23712055e-02, -4.53396541e-02,
        -1.23861394e-12]])